In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import minimize_scalar
from scipy.optimize import minimize 
from typing import List, Optional
import seaborn as sns

# =============================================================================
# Satellite Constellation Optimization with Multi-Constraints
# =============================================================================


In [ ]:
# Setup my utitlity functions

def great_circle_distance(lat1, lon1, lat2, lon2):
    """calculates great circle (angluar) distancce between two geographical coorindates"""
    lat1_rad, lon1_rad, lat2_rad, lon2_rad = map(np.radians, [lat1, lon1, lat2, lon2])

    x1 = np.cos(lat1_rad) * np.cos(lon1_rad)
    y1 = np.cos(lat1_rad) * np.sin(lon1_rad) 
    z1 = np.sin(lat1_rad)

    x2 = np.cos(lat2_rad) * np.cos(lon2_rad)
    y2 = np.cos(lat2_rad) * np.sin(lon2_rad) 
    z2 = np.sin(lat2_rad)

    # dot product of the two vectors is the cos of the angle between the two coordinates
    dot_product = x1*x2 + y1*y2 + z1*z2
    angular_distance = np.arccos(max(-1, min(1, dot_product))) # force beetween 0 and 1 for numeric stability

    return angular_distance

def spherical_coverage_overlap(lat1, lon1, lat2, lon2):
    """calulate the overlap to optimise coverage"""

    angular_distance = great_circle_distance(lat1, lon1, lat2, lon2)

    overlap = np.cos(angular_distance/2)

    return max(0,overlap)

def explonential_signal_decay(distance, decay_constant = 8000):
    """Create exponential decay function for the communication matrix"""
    communication_quality = np.exp(-distance/decay_constant)
    return communication_quality

# define how my constraint matricces are constructed

def constraint_matrix(satellite_positions, constraint_type):
    """derive the coverage matrix for n satellites"""

    # How many positions havve been entered?
    number_satellites = len(satellite_positions)
    
    # What are constraint functions are available?
    constraint_functions = {"redundancy": lambda overlap: overlap**2,
                            "efficiency": lambda overlap: 1 - overlap}
    
    # What is the shape of my constraint matrix?
    C = np.zeros((number_satellites, number_satellites))

    for i in range(number_satellites):
        for j in range(number_satellites):

            if i == j: 
                C[i,j] = 1
            else: 
                overlap = spherical_coverage_overlap(satellite_positions[i][0], satellite_positions[i][1], 
                                                    satellite_positions[j][0], satellite_positions[j][1])
                try:
                    C[i,j] = constraint_functions[constraint_type](overlap)
                except KeyError:
                    valid_options = list(constraint_functions.keys())
                    raise ValueError(f"Unknown constraint: {constraint_type}. Valid options: {valid_options}")

    return C


def communication_matrix(satellite_positions, altitudes = None, **kwargs):
    """derive the communication matrix for na satellites"""
    # unpack kwargs (future)
    # How many positions havve been entered?
    number_satellites = len(satellite_positions)
    
    # What is the shape of my constraint matrix?
    C = np.zeros((number_satellites, number_satellites))

    for i in range(number_satellites):
        for j in range(number_satellites):

            if i == j: 
                C[i,j] = 1
            else: 
                angular_distance = great_circle_distance(satellite_positions[i][0], satellite_positions[i][1], 
                                                    satellite_positions[j][0], satellite_positions[j][1])
                
                distance_km = 6371 * angular_distance

               # print(f"Distance Sat {i+1} - Sat {j+1}: {distance_km:.1f} km")
                communication_quality = explonential_signal_decay(distance_km)
                # print(f" communication quality: {communication_quality:.6f}")
                C[i,j] = communication_quality

    return C

def combined_matrix(satellite_positions, weights: dict, altitudes = None,  **kwargs):
    """combine my constraint matricies"""

    efficiency_matrix = constraint_matrix(satellite_positions, "efficiency")
    redundancy_matrix = constraint_matrix(satellite_positions, "redundancy")
    signal_matrix = communication_matrix(satellite_positions, altitudes, **kwargs)

    C = weights["efficiency"] * efficiency_matrix + weights["redundancy"] * redundancy_matrix + weights["communication"] * signal_matrix

    return C

def objective(satellitte_positions, weights:dict):
    """calculate my eigenvalues for optimisation"""
    #print(f"Received positions type: {type(satellitte_positions)}")
    #print(f"Recieved positions {satellitte_positions}")
    #print(f"Received weights: {weights}")
    np_positions = np.reshape(satellitte_positions,(-1,2))
    C = combined_matrix(np_positions, weights)

    eigenvalues = np.linalg.eigvals(C)

    optimal = -np.min(eigenvalues)


    return optimal

def optimise_satellite_positions(satellitte_positions:np.ndarray, weights: dict, method = "scipy", **kwargs) -> np.ndarray:
    """find the best positions for my satellites"""
    
    # Let's validate the weights
    required_keys = {"efficiency", "redundancy", "communication"}
    if set(weights.keys()) != required_keys:
        raise KeyError(f" weights must be specifically {required_keys}")
    
    tolerance = 0.001
    if sum(weights.values()) - 1.0 > tolerance:
        raise ValueError(f"weights must add to 1, these weights add to {sum(weights.values())}:.6f")


    flat_satellite_positions = satellitte_positions.flatten()
    bounds = [(-90,90), (-180,180)] * len(satellitte_positions)
    args = (weights)

    optimised_object = minimize(objective, flat_satellite_positions, args = args, method = 'L-BFGS-B', bounds= bounds)
    optimised_positions = np.reshape(optimised_object['x'],(-1,2))
    maximised_eignevalues = abs(optimised_object['fun'])
    print(optimised_object)

    return optimised_positions







In [ ]:
# visualise my satellite positions

def latlon_to_cartesian(lat, lon, radius=1.0):
    """
    Convert latitude/longitude to 3D Cartesian coordinates.
    
    Args:
        lat: Latitude in degrees (-90 to 90)
        lon: Longitude in degrees (-180 to 180) 
        radius: Sphere radius (default 1.0 for unit sphere)
    
    Returns:
        (x, y, z) coordinates
    """
    # Convert to radians
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    
    # Spherical to Cartesian conversion
    x = radius * np.cos(lat_rad) * np.cos(lon_rad)
    y = radius * np.cos(lat_rad) * np.sin(lon_rad)  
    z = radius * np.sin(lat_rad)
    
    return x, y, z

def visualize_satellites_3d(satellite_positions, title="Satellite Constellation"):
    """
    Simple 3D visualization of satellite positions around Earth.
    
    Parameters:
    satellite_positions: list of (x, y, z) tuples or numpy array of shape (n_sats, 3)
    """
    
    # Convert to numpy array if needed
    if isinstance(satellite_positions, list):
        sats = np.array(satellite_positions)
    else:
        sats = satellite_positions

    # convert to cartesian coordinates
    satellite_coords_3d = np.array([
    latlon_to_cartesian(row[0], row[1]) for row in sats
])

    # Create 3D plot
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    
    # Draw Earth as a sphere
    u, v = np.mgrid[0:2*np.pi:50j, 0:np.pi:25j]
    x_earth = np.cos(u)*np.sin(v)
    y_earth = np.sin(u)*np.sin(v)
    z_earth = np.cos(v)
    ax.plot_surface(x_earth, y_earth, z_earth, alpha=0.3, color='blue')
    
   # Plot satellites as red dots
    
    ax.scatter(satellite_coords_3d[:, 0], satellite_coords_3d[:, 1], satellite_coords_3d[:, 3]) 
               c='red', s=100, marker='o', label='Satellites'
    
    # Add satellite labels
    for i, (x, y, z) in enumerate(sats):
        ax.text(x, y, z, f'  Sat{i+1}', fontsize=9)
    
    # Set equal aspect ratio and labels
    ax.set_xlabel('X (Earth Radii)')
    ax.set_ylabel('Y (Earth Radii)')
    ax.set_zlabel('Z (Earth Radii)')
    ax.set_title(title)
    ax.legend()
    
    # Make it look nice
    ax.set_box_aspect([1,1,1])
    
    plt.show()

    # display my matricies and eignevalues for debugging and playing with the constellation
    """
NumPy Matrix Formatter for Satellite Constellation Analysis
=========================================================

Professional formatting tools for viewing efficiency, redundancy, 
and communication matrices during satellite optimization.
"""

def format_matrix_professional(matrix: np.ndarray, 
                             matrix_name: str = "Matrix",
                             satellite_names: Optional[List[str]] = None,
                             precision: int = 3,
                             highlight_diagonal: bool = True) -> None:
    """
    Professional matrix formatting for satellite analysis.
    
    Args:
        matrix: The numpy matrix to format
        matrix_name: Name to display (e.g., "Efficiency Matrix")
        satellite_names: List of satellite names for labeling
        precision: Decimal precision for display
        highlight_diagonal: Whether to highlight diagonal elements
    """
    print(f"\n{'='*60}")
    print(f"🛰️  {matrix_name.upper()}")
    print(f"{'='*60}")
    
    # Create satellite labels if not provided
    if satellite_names is None:
        satellite_names = [f"Sat_{i}" for i in range(matrix.shape[0])]
    
    # Create DataFrame for nice formatting
    df = pd.DataFrame(matrix, 
                     index=satellite_names, 
                     columns=satellite_names)
    
    # Set pandas display options for this matrix
    with pd.option_context('display.precision', precision,
                          'display.float_format', lambda x: f'{x:.{precision}f}'):
        print(df)
    
    # Matrix statistics
    print(f"\n📊 Matrix Statistics:")
    print(f"   Shape: {matrix.shape}")
    print(f"   Min value: {np.min(matrix):.{precision}f}")
    print(f"   Max value: {np.max(matrix):.{precision}f}")
    print(f"   Mean: {np.mean(matrix):.{precision}f}")
    
    if matrix.shape[0] == matrix.shape[1]:  # Square matrix
        print(f"   Trace (diagonal sum): {np.trace(matrix):.{precision}f}")
        eigenvals = np.linalg.eigvals(matrix)
        print(f"   Largest eigenvalue: {np.max(eigenvals):.{precision}f}")
        print(f"   Smallest eigenvalue: {np.min(eigenvals):.{precision}f}")

def format_matrix_compact(matrix: np.ndarray, 
                         matrix_name: str = "Matrix",
                         precision: int = 2) -> None:
    """
    Compact matrix formatting for quick inspection.
    """
    print(f"\n🔍 {matrix_name} ({matrix.shape[0]}×{matrix.shape[1]}):")
    
    # Use numpy's array printing with custom precision
    with np.printoptions(precision=precision, suppress=True, linewidth=100):
        print(matrix)
    
    # Quick stats on one line
    print(f"   Range: [{np.min(matrix):.{precision}f}, {np.max(matrix):.{precision}f}]  "
          f"Mean: {np.mean(matrix):.{precision}f}")

def format_matrix_heatmap(matrix: np.ndarray,
                         matrix_name: str = "Matrix",
                         satellite_names: Optional[List[str]] = None,
                         figsize: tuple = (8, 6)) -> None:
    """
    Create a heatmap visualization of the matrix.
    """
    if satellite_names is None:
        satellite_names = [f"Sat_{i}" for i in range(matrix.shape[0])]
    
    plt.figure(figsize=figsize)
    
    # Create heatmap with custom colormap
    if matrix_name.lower().startswith('eff'):
        cmap = 'Blues'  # Blue for efficiency
    elif matrix_name.lower().startswith('red'):
        cmap = 'Greens'  # Green for redundancy  
    elif matrix_name.lower().startswith('comm'):
        cmap = 'Oranges'  # Orange for communication
    else:
        cmap = 'viridis'
    
    sns.heatmap(matrix, 
                annot=True, 
                fmt='.3f',
                xticklabels=satellite_names,
                yticklabels=satellite_names,
                cmap=cmap,
                center=0 if np.min(matrix) < 0 else None)
    
    plt.title(f'{matrix_name} Heatmap')
    plt.tight_layout()
    plt.show()

def format_all_matrices(efficiency_matrix: np.ndarray,
                       redundancy_matrix: np.ndarray, 
                       communication_matrix: np.ndarray,
                       satellite_names: Optional[List[str]] = None,
                       show_heatmaps: bool = False) -> None:
    """
    Format all three constraint matrices together for comparison.
    """
    matrices = [
        (efficiency_matrix, "Efficiency Matrix", "💙"),
        (redundancy_matrix, "Redundancy Matrix", "💚"), 
        (communication_matrix, "Communication Matrix", "🧡")
    ]
    
    print("\n" + "="*80)
    print("🛰️  SATELLITE CONSTELLATION CONSTRAINT ANALYSIS")
    print("="*80)
    
    for matrix, name, emoji in matrices:
        print(f"\n{emoji} {name}")
        print("-" * 50)
        format_matrix_compact(matrix, name, precision=3)
        
        if show_heatmaps:
            format_matrix_heatmap(matrix, name, satellite_names)

def compare_matrix_eigenvalues(efficiency_matrix: np.ndarray,
                              redundancy_matrix: np.ndarray,
                              communication_matrix: np.ndarray,
                              weights: tuple = (0.4, 0.3, 0.3)) -> None:
    """
    Compare eigenvalues across all constraint matrices.
    """
    print(f"\n🔍 EIGENVALUE ANALYSIS")
    print("="*50)
    
    matrices = [
        (efficiency_matrix, "Efficiency"),
        (redundancy_matrix, "Redundancy"), 
        (communication_matrix, "Communication")
    ]
    
    eigenvalue_data = []
    
    for matrix, name in matrices:
        eigenvals = np.linalg.eigvals(matrix)
        eigenvals = np.sort(eigenvals)[::-1]  # Sort descending
        
        print(f"\n{name} Matrix Eigenvalues:")
        for i, val in enumerate(eigenvals):
            print(f"  λ{i+1}: {val:.4f}")
        
        eigenvalue_data.append((name, eigenvals))
    
    # Combined matrix eigenvalues
    w_eff, w_red, w_comm = weights
    combined = (w_eff * efficiency_matrix + 
                w_red * redundancy_matrix + 
                w_comm * communication_matrix)
    
    combined_eigenvals = np.linalg.eigvals(combined)
    combined_eigenvals = np.sort(combined_eigenvals)[::-1]
    
    print(f"\nCombined Matrix (weights: {weights}):")
    for i, val in enumerate(combined_eigenvals):
        print(f"  λ{i+1}: {val:.4f}")
    
    print(f"\n🎯 Optimization Objective: {np.max(combined_eigenvals):.4f}")

def debug_matrix_properties(matrix: np.ndarray, matrix_name: str) -> None:
    """
    Debug matrix properties to identify issues.
    """
    print(f"\n🔧 DEBUG: {matrix_name}")
    print("-" * 40)
    
    # Check basic properties
    print(f"Shape: {matrix.shape}")
    print(f"Data type: {matrix.dtype}")
    print(f"Has NaN values: {np.isnan(matrix).any()}")
    print(f"Has infinite values: {np.isinf(matrix).any()}")
    
    # Symmetry check
    is_symmetric = np.allclose(matrix, matrix.T)
    print(f"Is symmetric: {is_symmetric}")
    
    # Positive definiteness
    eigenvals = np.linalg.eigvals(matrix)
    is_positive_def = np.all(eigenvals > 0)
    is_positive_semi = np.all(eigenvals >= 0)
    print(f"Positive definite: {is_positive_def}")
    print(f"Positive semi-definite: {is_positive_semi}")
    
    # Condition number
    cond_number = np.linalg.cond(matrix)
    print(f"Condition number: {cond_number:.2e}")
    if cond_number > 1e12:
        print("  ⚠️  Matrix is ill-conditioned!")

# Example usage functions
def demo_matrix_formatting():
    """
    Demonstrate all formatting options with sample satellite data.
    """
    # Sample 4-satellite system
    satellite_names = ["ISS", "Hubble", "Starlink-1", "GPS-III"]
    
    # Create sample matrices (replace with your actual matrices)
    efficiency = np.array([
        [1.0, 0.7, 0.8, 0.6],
        [0.7, 1.0, 0.5, 0.9],
        [0.8, 0.5, 1.0, 0.7],
        [0.6, 0.9, 0.7, 1.0]
    ])
    
    redundancy = np.array([
        [1.0, 0.3, 0.2, 0.4],
        [0.3, 1.0, 0.5, 0.1],
        [0.2, 0.5, 1.0, 0.3],
        [0.4, 0.1, 0.3, 1.0]
    ])
    
    communication = np.array([
        [0.0, 0.8, 0.6, 0.7],
        [0.8, 0.0, 0.9, 0.5],
        [0.6, 0.9, 0.0, 0.8],
        [0.7, 0.5, 0.8, 0.0]
    ])
    
    # Show all formatting options
    print("DEMO: Professional Matrix Formatting")
    print("="*60)
    
    # Professional format
    format_matrix_professional(efficiency, "Efficiency Matrix", satellite_names)
    
    # Compact format
    format_matrix_compact(redundancy, "Redundancy Matrix")
    
    # All matrices together
    format_all_matrices(efficiency, redundancy, communication, satellite_names)
    
    # Eigenvalue comparison
    compare_matrix_eigenvalues(efficiency, redundancy, communication)
    
    # Debug properties
    debug_matrix_properties(efficiency, "Efficiency Matrix")

'''if __name__ == "__main__":
    demo_matrix_formatting()'''

'if __name__ == "__main__":\n    demo_matrix_formatting()'

In [20]:
# test for 3 satellites

my_satellites = np.array([
    [50.0, 10.0],    # Germany (Frankfurt area)
    [50.5, 10.3],    # ~50km northeast  
    [49.7, 9.8]      # ~50km southwest
])

satellite_positions_cartesian = np.zeros((len(my_satellites), 3))
for i in range(len(my_satellites)):
    satellite_positions_cartesian[i] = latlon_to_cartesian(my_satellites[i][0], my_satellites[i][1] )

# print(my_satellites[1][0])

z_vals = satellite_positions_cartesian[:,2]
print(satellite_positions_cartesian[:,2])
print(z_vals)

# Let's look at my satellites
    
visualize_satellites_3d(my_satellites, "My First Constellation")

weights = {"efficiency": 0, "redundancy": 0, "communication": 1}


best_spots = optimise_satellite_positions(my_satellites, weights)
    
# Let's look at m optimised satellite    
visualize_satellites_3d(best_spots, "Optimized 3-Satellite Configuration")





[0.76604444 0.77162458 0.76266833]
[0.76604444 0.77162458 0.76266833]


TypeError: int() argument must be a string, a bytes-like object or a real number, not 'tuple'

_______________________________________________________________________________________________________________________________

In [6]:
# Define constraints for optimisation

def coverage_matrix(theta):
    """Coverage quality matrix based on satellite separation angle theta."""
    overlap = np.cos(theta/2)  # More overlap when satellites are closer
    return np.array([[1.0, overlap],
                     [overlap, 1.0]])

def redundancy_matrix(theta):
    """Redundancy matrix - we WANT some overlap for failure tolerance."""
    overlap = np.cos(theta/2)
    redundancy = overlap**2  # Quadratic penalty for no redundancy
    return np.array([[redundancy, overlap],
                     [overlap, redundancy]])

def communication_matrix(theta):
    """Inter-satellite communication quality matrix."""
    # Communication quality decreases with distance
    # But too close satellites interfere with each other
    distance = abs(theta)
    if distance > np.pi:
        distance = 2*np.pi - distance  # Wrap around for shorter arc
    
    # Optimal communication at medium distances
    comm_quality = np.exp(-(distance - np.pi/2)**2)  # Peak at 90° separation
    return np.array([[0, comm_quality],
                     [comm_quality, 0]])

def combined_matrix(theta, w_coverage=0.6, w_redundancy=0.3, w_comm=0.1):
    """Weighted combination of all constraint matrices."""
    C_cov = coverage_matrix(theta)
    C_red = redundancy_matrix(theta)
    C_comm = communication_matrix(theta)
    
    return w_coverage * C_cov + w_redundancy * C_red + w_comm * C_comm


In [7]:

def objective_function(theta, weights=(0.6, 0.3, 0.1)):
    """Objective: maximize minimum eigenvalue of combined system."""
    C = combined_matrix(theta, *weights)
    eigenvals = np.linalg.eigvals(C)
    return -np.min(eigenvals)  # Negative because we want to maximize

def analyze_constellation():
    """Analyze how constellation performance varies with satellite separation."""
    
    # Grid search across all possible separations
    thetas = np.linspace(0, 2*np.pi, 1000)
    objectives = []
    min_eigenvals = []
    max_eigenvals = []
    
    for theta in thetas:
        C = combined_matrix(theta)
        eigenvals = np.linalg.eigvals(C)
        eigenvals = np.sort(eigenvals)
        
        objectives.append(-objective_function(theta))
        min_eigenvals.append(eigenvals[0])
        max_eigenvals.append(eigenvals[1])
    
    # Find optimal configuration
    optimal_idx = np.argmax(objectives)
    optimal_theta = thetas[optimal_idx]
    optimal_performance = objectives[optimal_idx]
    
    # Create comprehensive visualization
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Plot 1: Overall objective function
    ax1.plot(thetas * 180/np.pi, objectives, 'b-', linewidth=2, label='Combined Objective')
    ax1.axvline(optimal_theta * 180/np.pi, color='r', linestyle='--', 
                label=f'Optimal: {optimal_theta*180/np.pi:.1f}°')
    ax1.set_xlabel('Satellite Separation (degrees)')
    ax1.set_ylabel('System Performance')
    ax1.set_title('Multi-Constraint Optimization Result')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Eigenvalue spectrum
    ax2.plot(thetas * 180/np.pi, min_eigenvals, 'r-', linewidth=2, label='Min Eigenvalue')
    ax2.plot(thetas * 180/np.pi, max_eigenvals, 'g-', linewidth=2, label='Max Eigenvalue')
    ax2.axvline(optimal_theta * 180/np.pi, color='black', linestyle='--', alpha=0.7)
    ax2.set_xlabel('Satellite Separation (degrees)')
    ax2.set_ylabel('Eigenvalue')
    ax2.set_title('Eigenvalue Spectrum Analysis')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Plot 3: Individual constraint contributions
    coverage_objs = [-np.min(np.linalg.eigvals(coverage_matrix(theta))) for theta in thetas]
    redundancy_objs = [-np.min(np.linalg.eigvals(redundancy_matrix(theta))) for theta in thetas]
    comm_objs = [-np.min(np.linalg.eigvals(communication_matrix(theta))) for theta in thetas]
    
    ax3.plot(thetas * 180/np.pi, coverage_objs, 'b-', label='Coverage', linewidth=2)
    ax3.plot(thetas * 180/np.pi, redundancy_objs, 'orange', label='Redundancy', linewidth=2)
    ax3.plot(thetas * 180/np.pi, comm_objs, 'green', label='Communication', linewidth=2)
    ax3.axvline(optimal_theta * 180/np.pi, color='r', linestyle='--', alpha=0.7)
    ax3.set_xlabel('Satellite Separation (degrees)')
    ax3.set_ylabel('Individual Performance')
    ax3.set_title('Individual Constraint Analysis')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Plot 4: Configuration visualization
    ax4.set_xlim(-1.5, 1.5)
    ax4.set_ylim(-1.5, 1.5)
    
    # Draw Earth
    earth = plt.Circle((0, 0), 1, fill=False, color='blue', linewidth=3)
    ax4.add_patch(earth)
    
    # Draw optimal satellite positions
    sat1_x, sat1_y = 1.3 * np.cos(0), 1.3 * np.sin(0)
    sat2_x, sat2_y = 1.3 * np.cos(optimal_theta), 1.3 * np.sin(optimal_theta)
    
    ax4.plot(sat1_x, sat1_y, 'ro', markersize=10, label='Satellite 1')
    ax4.plot(sat2_x, sat2_y, 'go', markersize=10, label='Satellite 2')
    
    # Draw coverage areas (simplified)
    coverage1 = plt.Circle((sat1_x, sat1_y), 0.5, alpha=0.3, color='red')
    coverage2 = plt.Circle((sat2_x, sat2_y), 0.5, alpha=0.3, color='green')
    ax4.add_patch(coverage1)
    ax4.add_patch(coverage2)
    
    # Draw communication link
    ax4.plot([sat1_x, sat2_x], [sat1_y, sat2_y], 'k--', alpha=0.7, linewidth=2)
    
    ax4.set_aspect('equal')
    ax4.set_title(f'Optimal Configuration: {optimal_theta*180/np.pi:.1f}° Separation')
    ax4.legend()
    ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print optimization results
    print("🛰️ CONSTELLATION OPTIMIZATION RESULTS")
    print("=" * 50)
    print(f"Optimal satellite separation: {optimal_theta*180/np.pi:.1f}°")
    print(f"System performance score: {optimal_performance:.3f}")
    print(f"Performance at 0° (same position): {-objective_function(0):.3f}")
    print(f"Performance at 180° (opposite): {-objective_function(np.pi):.3f}")
    print(f"Performance at 90° (perpendicular): {-objective_function(np.pi/2):.3f}")
    
    # Analyze optimal configuration
    C_optimal = combined_matrix(optimal_theta)
    eigenvals_optimal = np.linalg.eigvals(C_optimal)
    print(f"\nOptimal configuration eigenvalues: {eigenvals_optimal}")
    print(f"Minimum eigenvalue (bottleneck): {np.min(eigenvals_optimal):.3f}")
    
    return optimal_theta, optimal_performance


In [11]:
analyze_constellation()

NameError: name 'analyze_constellation' is not defined